In [3]:
import time
from bs4 import BeautifulSoup
import yfinance as yf
import requests
import pandas as pd
import random
import utils

In [11]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.150 Safari/537.36"
}
def getProxy():
    list_=[
        "192.99.144.208:10001",
        "43.157.67.116:8888",
        "43.157.8.79:8888",
        "85.112.193.37:8080",
        "15.204.161.192:18080",
    ]
    p = {
        "https": random.choice(list_)
    }
    return p

In [9]:
def getHistoryData(id: str):
    try:
        stock = yf.Ticker(id)
        stockData = stock.history(period="max" ,raise_errors=True)
        return stockData
    except Exception as e:
        print(e)
        return None


def getHistoryEquityPreYear(stock_id: str):
    try:
        refer = {
            "Referer": F"https://goodinfo.tw/tw/StockAssetsStatus.asp?STOCK_ID={stock_id}"
        }
        response = requests.post(F"https://goodinfo.tw/tw/StockAssetsStatus.asp?STOCK_ID={stock_id}", headers={**headers, **refer})
        soup = BeautifulSoup(response.text, "html.parser")
        soup = soup.find("table", {"id": "tblDetail"})
        rows = soup.find_all("tr")
        data = {}
        for row in rows:
            try:
                cols = row.find_all("td")
                year = cols[0].text if 'Q' not in cols[0].text else "20" + cols[0].text.split("Q")[0]
                equity = int(str(cols[1].text.replace(",","")))
            except:
                continue
            data[year] = equity
        return data
    except Exception as e:
        print(e)
        return None

In [7]:

def run(stock_id = "2330"):
    print(F"start {stock_id}")
    df = getHistoryData(stock_id + ".TW")
    if df is None:
        return
    equity = getHistoryEquityPreYear(stock_id)
    if equity is None:
        return

    dataLs=[]
    for index, row in df.iterrows():
        data = {
            "date":str(row.name),
            "close":row["Close"],
            # "open":row["Open"],
            # "high":row["High"],
            # "low":row["Low"],
            # "volume":row["Volume"],
            # "dividends":row["Dividends"],
            # "stock splits":row["Stock Splits"],
            "equity":equity[str(row.name)[0:4]]*row["Close"] if  str(row.name)[0:4] in equity else None
        }
        dataLs.append(data)

    # out csv

    df = pd.DataFrame(dataLs)
    df.to_csv(F"data/stocks/{stock_id}TW.csv", index=False)
    

In [12]:
stocks = utils.get_all_stock(1101, 9999)
for stock in stocks:
    run(str(stock.stock_id))
    time.sleep(4)

start 1103
start 1104
start 1105
1105.TW: No timezone found, symbol may be delisted
start 1106
1106.TW: No timezone found, symbol may be delisted
start 1107
1107.TW: No price data found, symbol may be delisted (1d 1924-11-23 -> 2023-10-31)
start 1108
start 1109
start 1110
'NoneType' object has no attribute 'find_all'
start 1111
1111.TW: No timezone found, symbol may be delisted
start 1112
1112.TW: No timezone found, symbol may be delisted
start 1113
1113.TW: No timezone found, symbol may be delisted
start 1114
1114.TW: No timezone found, symbol may be delisted
start 1115
1115.TW: No timezone found, symbol may be delisted
start 1116
1116.TW: No timezone found, symbol may be delisted
start 1117
1117.TW: No timezone found, symbol may be delisted
start 1118
1118.TW: No timezone found, symbol may be delisted
start 1119
1119.TW: No timezone found, symbol may be delisted
start 1120
1120.TW: No timezone found, symbol may be delisted
start 1121
1121.TW: No timezone found, symbol may be delisted

KeyboardInterrupt: 